In [ ]:
from abc import ABC, abstractmethod
from typing import List
import datetime
from datetime import timedelta

import threading

class Argentur:
    pass
class Servicio:
    pass
class Itinerario:
    pass
class Ciudad:
    pass
class Reserva:
    pass
class Unidad:
    pass
class Asiento:
    pass
class Pasajero:
    pass
class Venta:
    pass
class MedioPago(ABC):
    pass
class TarjetaCredito(MedioPago):
    pass
class MercadoPago(MedioPago):
    pass
class Uala(MedioPago):
    pass

class Argentur:
    def __init__(self):
        self.sistema_activo: bool = True
        self.itinerarios: List[Itinerario] = []
        self.servicios: List[Servicio] = []
        self.ventas: List[Venta] = []

    def crear_itinerario(self, origen: Ciudad, destino: Ciudad, paradas_intermedias: List[Ciudad] = None):
        nuevo_itinerario = Itinerario(origen, destino, paradas_intermedias)
        self.itinerarios.append(nuevo_itinerario)

    def crear_servicio(self, num_servicio: int, unidad: Unidad, precio: float, calidad: str, fecha_partida: datetime.datetime, fecha_llegada: datetime.datetime, itinerario: Itinerario):
        nuevo_servicio = Servicio(num_servicio, unidad, precio, calidad, fecha_partida, fecha_llegada, itinerario)
        self.servicios.append(nuevo_servicio)

    def crear_reserva(self, num_servicio: int, pasajero: Pasajero, fecha_reserva: datetime.datetime, num_asiento: int) -> bool:
        servicio = self.obtener_servicio(num_servicio)
        if servicio is None:
            return None
        return servicio.crear_reserva(pasajero, fecha_reserva, num_asiento)

    def crear_venta(self, servicio: Servicio, pasajero: Pasajero, num_asiento: int, medio_pago: MedioPago):
        for venta in self.ventas:
            if venta.obtener_num_servicio() == servicio and venta.obtener_asiento() == num_asiento:
                return None
        fecha_venta = datetime.datetime.today()
        print(medio_pago.validar_tarjeta())
        servicio.cancelar_reserva(num_asiento)
        venta = Venta(servicio, pasajero, num_asiento, medio_pago, fecha_venta)
        self.ventas.append(venta)
        servicio.reservar_asiento(num_asiento)
        return Venta

    def calcular_total(self, fecha_desde: datetime.datetime, fecha_hasta: datetime.datetime):
        total_facturado = 0
        ventas_en_rango = [venta for venta in self.ventas if fecha_desde <= venta.obtener_fecha() <= fecha_hasta]
        for venta in ventas_en_rango:
            total_facturado += venta.obtener_precio()
        return total_facturado

    def calcular_ventas_por_destino_y_medio_pago(self, fecha_desde: datetime.datetime, fecha_hasta: datetime.datetime):
        cantidad_por_destino = {}
        cantidad_por_medio_pago = {}
        ventas_en_rango = [venta for venta in self.ventas if fecha_desde <= venta.obtener_fecha() <= fecha_hasta]
        for venta in ventas_en_rango:
            destino = venta.obtener_destino()
            medio_pago = venta.obtener_medio_pago()
            cantidad_por_destino[destino] = cantidad_por_destino.get(destino, 0) + 1
            cantidad_por_medio_pago[medio_pago] = cantidad_por_medio_pago.get(medio_pago, 0) + 1
        return [cantidad_por_destino, cantidad_por_medio_pago]

    def obtener_itinerarios(self):
        return self.itinerarios

    def obtener_servicios(self):
        return self.servicios

    def obtener_servicio(self, num_servicio: int):
        aux = None
        for servicio in self.servicios:
            if servicio.obtener_num_servicio() == num_servicio:
                aux = servicio
        return aux

    def obtener_ventas(self):
        return self.ventas

class Servicio:
    def __init__(self, num_servicio: int, unidad: Unidad, precio: float, calidad: str, fecha_partida: datetime.datetime, fecha_llegada: datetime.datetime, itinerario: Itinerario):
        self.num_servicio = num_servicio
        self.unidad = unidad
        self.precio = precio
        self.calidad = calidad
        self.fecha_partida = fecha_partida
        self.fecha_llegada = fecha_llegada
        self.itinerario = itinerario
        self.reservas: List[Reserva] = []

        self.schedule_cancelar_reservas()

    def crear_reserva(self,  pasajero: Pasajero, fecha_reserva: datetime.datetime, num_asiento: int):
        self.reservar_asiento(num_asiento)
        nueva_reserva = Reserva(self.num_servicio, pasajero, fecha_reserva, num_asiento)
        self.reservas.append(nueva_reserva)
        return nueva_reserva

    def cancelar_reserva(self, num_asiento: int):
        for reserva in self.reservas:
            if reserva.obtener_asiento() == num_asiento:
                self.unidad.cancelar_reserva(num_asiento)
                self.reservas.remove(reserva)
                return True
        return False

    def schedule_cancelar_reservas(self):
        ahora = datetime.datetime.now()
        delay = (self.fecha_partida - ahora + timedelta(minutes=30)).total_seconds()
        delay = max(0, delay)
        timer = threading.Timer(delay, self.cancelar_reservas_pasadas_de_tiempo)
        timer.start()

    def cancelar_reservas_pasadas_de_tiempo(self):
        ahora = datetime.datetime.now()
        if self.fecha_partida - ahora <= timedelta(minutes=30):
          # Crea una copia de la lista de reservas para evitar errores en la iteracion al eliminar items de esta
          reservas_a_cancelar = self.reservas.copy()
          for reserva in reservas_a_cancelar:
            self.cancelar_reserva(reserva.obtener_asiento())

    def consultar_disponibilidad(self, num_asiento: int) -> bool:
        return self.unidad.obtener_disponibilidad_asiento(num_asiento)

    def obtener_num_servicio(self):
        return self.num_servicio

    def obtener_origen(self):
        return self.itinerario.obtener_origen()

    def obtener_destino(self):
        return self.itinerario.obtener_destino()

    def obtener_paradas_intermedias(self):
        return self.itinerario.obtener_paradas_intermedias()

    def obtener_fecha_partida(self):
        return self.fecha_partida

    def obtener_fecha_llegada(self):
        return self.fecha_llegada

    def obtener_precio(self):
        return self.precio

    def obtener_calidad(self):
        return self.calidad

    def obtener_asientos_libres(self):
        return self.unidad.obtener_asientos_libres()

    def obtener_asientos_ocupados(self):
        return self.unidad.obtener_asientos_ocupados()

    def reservar_asiento(self, num_asiento: int):
        return self.unidad.reservar_asiento(num_asiento)

    def obtener_reservas(self):
        return self.reservas

class Itinerario:
    def __init__(self, origen: Ciudad, destino: Ciudad, paradas_intermedias: List[Ciudad] = None):
        self.origen = origen
        self.destino = destino
        self.paradas_intermedias = paradas_intermedias

    def obtener_origen(self):
        return self.origen.obtener_nombre()

    def obtener_destino(self):
        return self.destino.obtener_nombre()

    def obtener_paradas_intermedias(self):
        return [ciudad.obtener_nombre() for ciudad in self.paradas_intermedias]

class Ciudad:
    def __init__(self, nombre: str, codigo: str, provincia: str):
        self.codigo = codigo
        self.nombre = nombre
        self.provincia = provincia

    def obtener_nombre(self):
        return self.nombre

class Reserva:
    def __init__(self, num_servicio: int,pasajero: Pasajero, fecha_reserva: datetime.datetime, num_asiento: int):
        self.num_servicio = num_servicio
        self.pasajero = pasajero
        self.fecha_reserva = fecha_reserva
        self.num_asiento = num_asiento

    def obtener_pasajero(self):
        return self.pasajero.obtener_nombre()

    def obtener_asiento(self):
        return self.num_asiento

    def obtener_FechaHora_reserva(self):
        return self.fecha_reserva

class Venta:
    def __init__(self, servicio: Servicio, pasajero: Pasajero, num_asiento: int, medio_pago: MedioPago, fecha_venta: datetime.datetime):
        self.servicio = servicio
        self.pasajero = pasajero
        self.num_asiento = num_asiento
        self.medio_pago = medio_pago
        self.fecha_venta = fecha_venta

    def obtener_num_servicio(self):
        return self.servicio.obtener_num_servicio()

    def obtener_fecha(self):
        return self.fecha_venta

    def obtener_medio_pago(self):
        return self.medio_pago.obtener_nombre()

    def obtener_precio(self):
        return self.servicio.obtener_precio()

    def obtener_destino(self):
        return self.servicio.obtener_destino()

class Unidad:
    def __init__(self, patente: str, num_asientos : int):
        self.patente = patente
        self.asientos: List[Asiento] = []
        for i in range(num_asientos):
            self.asientos.append(Asiento(i + 1))

    def obtener_asientos_libres(self):
        return [asiento.obtener_numero_libre() for asiento in self.asientos if asiento.obtener_numero_libre() is not None]

    def obtener_asientos_ocupados(self):
        return [asiento.obtener_numero_ocupado() for asiento in self.asientos if asiento.obtener_numero_ocupado() is not None]

    def reservar_asiento(self, num_asiento: int):
        self.asientos[num_asiento - 1].reservar_asiento()

    def cancelar_reserva(self, num_asiento: int):
        return self.asientos[num_asiento - 1].establecer_estado(False)

    def obtener_disponibilidad_asiento(self, num_asiento: int):
        if num_asiento < 1 or num_asiento > len(self.asientos):
            return False
        return not self.asientos[num_asiento - 1].obtener_estado()

class Asiento:
    def __init__(self, numero: int):
        self.numero = numero
        self.ocupado = False

    def obtener_numero_libre(self):
        if self.ocupado == False:
            return self.numero

    def obtener_numero_ocupado(self):
        if self.ocupado:
            return self.numero

    def establecer_estado(self, estado: bool):
        aux = self.ocupado
        self.ocupado = estado
        return aux != estado

    def obtener_estado(self):
        return self.ocupado

    def reservar_asiento(self):
        if self.ocupado == False:
            self.ocupado = True


class Pasajero:
    def __init__(self, nombre: str, email: str, dni: str):
        self.nombre = nombre
        self.email = email
        self.dni = dni

    def obtener_nombre(self):
        return self.nombre

class MedioPago(ABC):
    @abstractmethod
    def obtener_nombre(self):
        pass

    @abstractmethod
    def validar_tarjeta(self):
        pass

class TarjetaCredito(MedioPago):
    def __init__(self, numero: str, dni_titular: str, nombre: str, fecha_vencimiento: datetime.datetime):
        self.numero = numero
        self.dni_titular = dni_titular
        self.nombre = nombre
        self.fecha_vencimiento = fecha_vencimiento

    def obtener_nombre(self):
        return "Tarjeta de credito"

    def validar_tarjeta(self):
        return "Tarjeta de credito validada correctamente"

class MercadoPago(MedioPago):
    def __init__(self, celular: str, email: str):
        self.celular = celular
        self.email = email

    def obtener_nombre(self):
        return "Mercado pago"

    def validar_tarjeta(self):
        return "Tarjeta de mercado pago validada correctamente"

class Uala(MedioPago):
    def __init__(self, email: str, nombre_titular: str):
        self.email = email
        self.nombre_titular = nombre_titular

    def obtener_nombre(self):
        return "Uala"

    def validar_tarjeta(self):
        return "Tarjeta uala validada correctamente"

#------------------------------------------------------------------------------------------------------------------------

sistema: Argentur = Argentur()
u1 = Unidad("ABC123", 50)
u2 = Unidad("DEF456", 50)
c1 = Ciudad("Buenos Aires", "BA", "Buenos Aires")
c2 = Ciudad("Córdoba", "CD", "Córdoba")
c3 = Ciudad("Mendoza", "MZ", "Mendoza")
c4 = Ciudad("Salta", "SLT", "Salta")
c5 = Ciudad("San Luis", "SL", "San Luis")
c6 = Ciudad("Rosario", "RS", "Santa Fe")
sistema.crear_itinerario(c1, c2, [c6])
sistema.crear_itinerario(c2, c3, [c5])
sistema.crear_itinerario(c3, c4)
itinerarios = sistema.obtener_itinerarios()
sistema.crear_servicio(124334, u1, 1000, "Económico", datetime.datetime(
    2023, 10, 1, 10, 0), datetime.datetime(2023, 10, 1, 12, 0), itinerarios[0])
sistema.crear_servicio(2, u2, 1500, "Premium", datetime.datetime(
    2023, 10, 2, 10, 0), datetime.datetime(2023, 10, 2, 12, 0), itinerarios[1])
sistema.crear_servicio(1, u2, 1500, "Premium", datetime.datetime.now(
) - timedelta(minutes=28), datetime.datetime(2023, 10, 2, 12, 0), itinerarios[1])

def menu():
    print("Sistema de reservas ArgenTour")
    print("Menu principal")
    print("1. Listar servicios")
    print("2. Crear Reserva")
    print("3. Crear Venta")
    print("4. Crear Informe")
    print("-1. Salir")
    opcion = int(input("Seleccionar una opcion: "))
    if opcion == 1:
        listar_servicios()
    elif opcion == 2:
        crear_reserva()
    elif opcion == 3:
        crear_venta()
    elif opcion == 4:
        generar_informe()
    elif opcion == -1:
        return
    else:
        print("Opcion no valida")
    menu()

def listar_servicios():
    print("Servicios disponibles:")
    for servicio in sistema.obtener_servicios():
        print(f"Servicio: {servicio.obtener_num_servicio()}")
        print("Itinerario:")
        print(f"Origen: {servicio.obtener_origen()}")
        print(f"Paradas intermedias: {','.join(servicio.obtener_paradas_intermedias())}")
        print(f"Destino: {servicio.obtener_destino()}")
        print(f"Calidad: {servicio.obtener_calidad()}")
        print(f"Precio: {servicio.obtener_precio()}")
        print(f"Fecha de partida: {servicio.obtener_fecha_partida()}")
        print(f"Fecha de llegada: {servicio.obtener_fecha_llegada()}")
        print("-" * 20)

def crear_reserva():
    print("Crear reserva")
    servicio = menu_seleccion_servicio()
    asiento = menu_seleccion_asiento(servicio)
    pasajero = menu_datos_pasajero()
    fecha_reserva = datetime.datetime.today()
    reserva = sistema.crear_reserva(servicio.obtener_num_servicio(), pasajero, fecha_reserva, asiento)
    menu_reserva(reserva, servicio, pasajero, asiento)

def crear_venta():
    print("Crear venta")
    servicio = menu_seleccion_servicio()
    print("1. Vender asiento reservado")
    print("2. Vender asiento no reservado")
    opcion = int(input("Seleccione una opcion: "))
    if opcion == 1:
        menu_venta_asiento_reservado(servicio)
    elif opcion == 2:
        menu_venta_asiento_no_reservado(servicio)
    else:
        print("Opcion no valida")
    print("-" * 20)

def generar_informe():
    print("Generar informe")
    fechas = menu_seleccion_fechas()
    total = sistema.calcular_total(fechas[0], fechas[1])
    cantidades = sistema.calcular_ventas_por_destino_y_medio_pago(fechas[0],fechas[1])
    menu_informe(total, cantidades)

def menu_venta_asiento_reservado(servicio: Servicio):
    for reserva in servicio.obtener_reservas():
        print(f"Reserva: {reserva.obtener_pasajero()} - Asiento: {reserva.obtener_asiento()}")
    num_asiento = int(input("Ingrese el numero de asiento: "))
    while servicio.consultar_disponibilidad(num_asiento):
        print("El asiento no esta reservado")
        num_asiento = int(input("Ingrese el numero de asiento: "))
    pasajero = None
    for reserva in servicio.obtener_reservas():
        if reserva.obtener_asiento() == num_asiento:
            pasajero = reserva.obtener_pasajero()
            break
    if pasajero is None:
        print("No se encontro la reserva")
        return
    medio_pago = menu_medio_de_pago()
    if medio_pago is None:
        print("No se selecciono medio de pago")
        return
    if sistema.crear_venta(servicio, pasajero, num_asiento, medio_pago):
        print(f"Venta realizada: Pasajero: {reserva.obtener_pasajero()}, Asiento: {num_asiento}, Medio de pago: {medio_pago.obtener_nombre()}")
    else:
        print("No se pudo crear la venta")

def menu_venta_asiento_no_reservado(servicio: Servicio):
    num_asiento = menu_seleccion_asiento(servicio)
    pasajero = menu_datos_pasajero()
    if pasajero is None:
        return
    medio_pago = menu_medio_de_pago()
    if medio_pago is None:
        return
    venta = sistema.crear_venta(servicio, pasajero, num_asiento, medio_pago)
    if venta:
        print(f"Venta realizada: Pasajero: {pasajero.obtener_nombre()}, Asiento: {num_asiento}, Medio de pago: {medio_pago.obtener_nombre()}")
    else:
        print("No se pudo crear la venta")
    print("-" * 20)

def menu_medio_de_pago():
    print("Ingrese metodo de pago")
    print("1. Tarjeta de credito")
    print("2. Mercado pago")
    print("3. Uala")
    opcion = input()
    if opcion == "1":
        nro_tc = input("Ingrese numero de tarjeta de credito: ")
        nom_t = input("Ingrese nombre del titular: ")
        dni_t = input("Ingrese dni del titular: ")
        fecha_vencimiento = datetime.datetime.strptime(input("Ingrese fecha de vencimiento (YYYY-MM):"),"%Y-%m")
        mp = TarjetaCredito(nro_tc, dni_t, nom_t, fecha_vencimiento)
    elif opcion == "2":
        nro_celular = input("Ingrese numero de celular: ")
        email = input("Ingrese email: ")
        mp = MercadoPago(nro_celular, email)
    elif opcion == "3":
        email = input("Ingrese email: ")
        nom_titular = input("Ingrese nombre del titular: ")
        mp = Uala(email, nom_titular)
    else:
        print("Opcion no valida")
        return
    return mp

def menu_seleccion_servicio():
    num_servicio = int(input("Ingrese el numero de servicio: "))
    while sistema.obtener_servicio(num_servicio) is None:
        print("Numero de servicio no valido")
        num_servicio = int(input("Ingrese el numero de servicio: "))
    servicio = sistema.obtener_servicio(num_servicio)
    return servicio

def menu_seleccion_asiento(servicio: Servicio):
    print(f"Asientos libres: {servicio.obtener_asientos_libres()}")
    print(f"Asientos ocupados: {servicio.obtener_asientos_ocupados()}")
    num_asiento = int(input("Ingrese el numero de asiento: "))
    while not servicio.consultar_disponibilidad(num_asiento):
        print("El asiento no esta disponible")
        num_asiento = int(input("Ingrese el numero de asiento: "))
    servicio.reservar_asiento(num_asiento)
    return num_asiento

def menu_datos_pasajero():
    nombre_pasajero = input("Ingrese el nombre del pasajero: ")
    email_pasajero = input("Ingrese el email del pasajero: ")
    dni_pasajero = input("Ingrese el dni del pasajero: ")
    pasajero = Pasajero(nombre_pasajero, email_pasajero, dni_pasajero)
    return pasajero

def menu_reserva(reserva: Reserva, servicio: Servicio, pasajero: Pasajero, asiento: Asiento):
    if reserva != None:
        print(f"Reserva realizada: Pasajero {pasajero.obtener_nombre()}, Asiento {asiento}, servicio del {servicio.obtener_fecha_partida()}")
        print(f"Asientos libres: {servicio.obtener_asientos_libres()}")
        print(f"Asientos ocupados: {servicio.obtener_asientos_ocupados()}")
    else:
        print("No se pudo crear la reserva")

def menu_seleccion_fechas():
    print("Ingrese fecha desde (YYYY-MM-DD): ")
    fecha_desde_str = input()
    fecha_desde = datetime.datetime.strptime(fecha_desde_str, "%Y-%m-%d")
    print("Ingrese fecha hasta (YYYY-MM-DD): ")
    fecha_hasta_str = input()
    fecha_hasta = datetime.datetime.strptime(fecha_hasta_str, "%Y-%m-%d")
    print("-" * 20)
    return [fecha_desde,fecha_hasta]

def menu_informe(total, cantidades):
    print(f"Total facturado: {total}")
    print("Cantidad de ventas por destino:")
    for destino, cantidad in cantidades[0].items():
        print(f"{destino}: {cantidad}")
    print("Cantidad de ventas por medio de pago:")
    for medio_pago, cantidad in cantidades[1].items():
        print(f"{medio_pago}: {cantidad}")
    print("-" * 20)

#----------------------------------------------------------------------------------------------------------------

if __name__ == "__main__":
    menu()